In [ ]:
import sys
sys.path.append('src')

import pandas as pd
import numpy as np
import itertools
import random
import time
import swifter

from src.SABR_IV_approximators import AntonovApprox #, AntonovExact
from src import helperfunctions as hf

### Parameter ranges as 4.3.2

In [ ]:
days = [7, 14, 21, 30, 60, 90, 180, 270]
years = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
alpha_range = np.linspace(.05, 1, 20)
rho_range = np.linspace(-.95, .5, 20)

K_range = np.arange(0.4,1.7,0.1)
K_range = np.round(K_range,1)

# Fixed grid parameter sampling

#### First short term <1 year

In [ ]:
params = ['alpha', 'rho','v', 'T', 'K']
for day in days:
    T = day/365
    v_range = np.linspace(hf.v_t_lower(T), hf.v_t_upper(T), 20)
    string = str(day) + 'd'
    comb = [alpha_range, rho_range, v_range, [T], K_range]
    res = list(itertools.product(*comb))
    res = pd.DataFrame(res, columns=params)
    
    start = time.time()
    res['impl_volatility'] = res.swifter.apply(lambda x: AntonovApprox().calc_iv(x[0], 0.5, x[1], x[2], (x[4], 1), x[3]), axis=1)
    stop = time.time()
    print(stop-start)
    
    res.to_pickle('training_data/train_' + string + '.pkl')

#### Now long term >1 year

In [ ]:
params = ['alpha', 'rho','v', 'T', 'K']
for year in years:
    T = year
    v_range = np.linspace(hf.v_t_lower(T), hf.v_t_upper(T), 20)
    string = str(year) + 'y'
    comb = [alpha_range, rho_range, v_range, [T], K_range]
    res = list(itertools.product(*comb))
    res = pd.DataFrame(res, columns=params)
    
    start = time.time()
    res['impl_volatility'] = res.swifter.apply(lambda x: AntonovApprox().calc_iv(x[0], 0.5, x[1], x[2], (x[4], 1), x[3]), axis=1)
    stop = time.time()
    print(stop-start)
    
    
    res.to_pickle('training_data/train_' + string + '.pkl')

## Random sampling strikes
k = 1e5

In [ ]:
#random sampling
k = 100000
for i in range(1,50):
    alpha_random = np.random.uniform(0, 1, size=k)
    rho_random = np.random.uniform(-.999, .999, size=k)
    K_random = np.random.uniform(0.4, 1.6, size=k)

    if i % 2 == 0:
        T_random = np.random.uniform(1, 10, size=k)
        # sample random v from variable range
        v_random = [np.random.uniform(hf.v_t_lower(t), hf.v_t_upper(t)) for t in T_random]
    else:
        T_random = np.random.uniform(7/365, 1, size=k)
        # sample random v from variable range
        v_random = [np.random.uniform(hf.v_t_lower(t), hf.v_t_upper(t)) for t in T_random]

    params = ['alpha', 'rho','v', 'T', 'K']
    res = pd.DataFrame(columns=params)
    res['alpha'] = alpha_random
    res['rho'] = rho_random
    res['v'] = v_random
    res['T'] = T_random
    res['K'] = K_random

    start = time.time()
    res['impl_volatility'] = res.swifter.apply(lambda x: AntonovApprox().calc_iv(x[0], 0.5, x[1], x[2], (x[4], 1), x[3]), axis=1)
#     res['impl_volatility'] = res.apply(lambda x: AntonovApprox().calc_iv(x[0], 0.5, x[1], x[2], (x[4], 1), x[3],0), axis=1)
    stop = time.time()
    print(stop - start)
    
    res.to_pickle('training_data/random_unstacked_' + str(i) + '.pkl')



# Aggregate all data

In [ ]:
training_data = pd.DataFrame()
for day in days:
    string = str(day) + 'd'
    df = pd.read_pickle('training_data/train_' + string + '.pkl')
    training_data = training_data.append(df)

for year in years:
    string = str(year) + 'y'
    df = pd.read_pickle('training_data/train_' + string + '.pkl')
    training_data = training_data.append(df)

In [ ]:
random_data = pd.DataFrame()
for i in range(1,33):
    df = pd.read_pickle('training_data/random_unstacked_' + str(i) + '.pkl')
    random_data = random_data.append(df)

In [ ]:
training_data = training_data[training_data['impl_volatility'] > 0]

In [ ]:
random_data = random_data[random_data['impl_volatility'] < 1000]

In [ ]:
training_data.to_pickle(r'C:\Users\hugo\OneDrive\Documents\Quantitative Finance\Thesis\Data\training_data\training_data.pkl')

In [ ]:
len(random_data)

In [ ]:
len(random_data[random_data['impl_volatility'] > 0])

In [ ]:
random_data = random_data[random_data['impl_volatility'] > 0]

In [ ]:
random_data.to_pickle(r'C:\Users\hugo\OneDrive\Documents\Quantitative Finance\Thesis\Data\training_data\test_data.pkl')